In [1]:
from tqdm.notebook import tqdm

filename = "../data/clean/bahan baku bikin kalimat.xlsx"
filename_spelling = "../data/clean/dict spelling rev.xlsx"
filename_kata = "../data/clean/skrip_kata_warung_final.xlsx"

In [2]:
import pandas as pd
df = pd.read_excel(filename)
df_dict = pd.read_excel(filename_spelling)
df_kata = pd.read_excel(filename_kata)

/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
product = df['Product Variant'].dropna()
group = df_dict.Group.dropna()
spell = df_dict.Spell.dropna()
numeric = df.NUMERIC.dropna()
uom = df['Quantity Variant'].dropna()

In [4]:
array_group = []
array_spell = []
array_numeric = []
array_product = []
array_product_delimiter = []

for i in group:
    if i != '@':
        array_group.append(i)
for i in spell:
    array_spell.append(i)
for i in numeric:
    array_numeric.append(i)
for i in product:
    array_product_delimiter.append(i)
    
product = product[product!='@']
product = product.reset_index(drop=True)    
for i in product:
    array_product.append(i.strip())

In [5]:
join_product = ','.join(array_product_delimiter)
join_spell = ','.join(array_spell)
join_uom = ','.join(uom)

In [6]:
spell = join_spell.split('@')
uom = join_uom.split('@')
prod = join_product.split('@')
spell_word = []
uom_word = []
prod_word = []

for i in spell:
    spell_word.append(i.strip(','))
for i in uom:
    uom_word.append(i.strip(','))
for i in prod:
    prod_word.append(i.strip(','))

In [7]:
# Total product which have this UOM type
length_product = []

for data in prod_word:
    length_product.append(len(data.split(',')))

In [8]:
## New UOM
new_uom = []

for idx in range (len(length_product)):
    for i in range (length_product[idx]):
        new_uom.append(uom_word[idx])

In [9]:
data = []

for line in array_product:        
    status = False
    for group in array_group:
        if group in line:
            status = True
            after_word = spell_word[array_group.index(group)]
            temp_uom = new_uom[array_group.index(group)]
            
            for val_spell in after_word.split(','):
                for val_uom in temp_uom.split(','):
                    for num in numeric:
                        data.append(line.replace(group.strip(), val_spell)+' '+num+' '+val_uom)
                        
    if status == False:
        temp_uom = new_uom[array_product.index(line)]
        for val_uom in temp_uom.split(','):
            for num in numeric:
                data.append(line+' '+num+' '+val_uom)
                
data = [i.strip() for i in data]

In [10]:
for i in range(3):
    for idx in range(0,len(data)):
        for group in array_group:
            if group in data[idx]:
                after_word = spell_word[array_group.index(group)]
            
                for val_spell in after_word.split(','):
                    data[idx] = data[idx].replace(group.strip(), val_spell)

In [11]:
for idx in range(0,len(data)):
    data[idx] = ' '.join(data[idx].split())

In [12]:
import random
random.choice(data)

'Larutan Kaki Tiga Botol Kecil Enem Buah'

In [52]:
with open('../data/clean/data.txt','w') as outfile:
    for line in data:
        outfile.write('%s\n' % line)